# Difftest：ISA 协同仿真框架

**Difftest: ISA Co-simulation framework**

有 NEMU 作为参考模型，我们可以尝试解决“仿真程序怎么知道自己已经出错了”和“找到错误发生的位置”这两个问题：我们可以通过同时运行 RTL 仿真和参考模型，实时比对两者的体系结构状态，从而发现仿真程序中的错误。

Now we have NEMU as the reference model, we can try to address the two issues of "How does the simulation program know it has already encountered an error" and "How to locate where the error occurred": We can run the RTL simulation and the reference model simultaneously, comparing their architectural states in real time to identify errors in the simulation program.

![difftest-arch](../images/02-functional/02-difftest/difftest-arch-en.png)

在运行 emu 时，通过 `--diff <path/to/ref.so>` 参数指定参考模型的动态链接库路径，即可启用 Difftest 功能。

When running emu, enable the DiffTest feature by specifying the path to the reference model's dynamic link library with the `--diff <path/to/ref.so>` parameter.

我们在启用 diff 的情况下再次运行 01-basics/04-run 中有错误的仿真程序：

We rerun the faulty emu from 01-basics/04-run with diff enabled:

In [ ]:
%%bash
source ../env.sh
cd ${NOOP_HOME}

$(get_asset emu-precompile/emu-alu-err) \
    -i $(get_asset workload/hello-riscv64-xs.bin) \
    --diff $(get_asset emu-precompile/riscv64-nemu-interpreter-so) \
    2>/dev/null

可以看到执行 pc = 0x0080000078 处的指令时，REF 和 DUT 执行产生了不一致，DUT 中 a0 值为 0，而 REF 为 0x2000。

At PC 0x0080000078, the REF and DUT are not matched: a0 is 0x2000 in the REF, but 0 in the DUT.

Difftest 在遇到错误时还会打印一些有用的信息，例如体系结构寄存器（整数/浮点/向量/CSR）的值。

Difftest also prints useful information when an error is encountered, such as the values of architectural registers (integer/floating-point/vector/CSR).

以及指令 commit 的顺序和其它信息。

And the order of instruction commits and other information.

在 Difftest 检测到错误后，我们可以再次运行仿真，这次在 difftest 报告的出错周期数附近启用波形输出。

After Difftest detects an error, we can rerun the simulation and enable waveform output around the failing cycle reported by Difftest.

In [ ]:
%%bash
source ../env.sh
cd ${NOOP_HOME}

mkdir -p build
rm -f ./build/*.vcd

$(get_asset emu-precompile/emu-alu-err) \
    -i $(get_asset workload/hello-riscv64-xs.bin) \
    --diff $(get_asset emu-precompile/riscv64-nemu-interpreter-so) \
    -b 8000 \
    -e 10000 \
    --dump-wave \
    2>/dev/null || true

echo -n "Dump wave: "
realpath ./build/*.vcd